In [150]:
import math
import pandas as pd
import time
import random
import csv

In [151]:
# Create an empty DataFrame
# df = pd.DataFrame(columns=["Test_Case", "Number_of_Points", "Execution Time", "Execution Time on CPU"])
# df.to_csv("Testfile_Quickhull.csv", index = False)

In [152]:
# Open and read the CSV file
with open('Testfile_Quickhull.csv', 'r') as csvfile:
    
    reader = csv.DictReader(csvfile)
    
    # Initialize a list to store the run numbers
    run_numbers = []
    
    # Iterate through the rows and extract the run numbers
    for row in reader:
    
        run_number = int(row['Test_Case'])
        run_numbers.append(run_number)


if len(run_numbers) == 0:
    
    last_run_number = 0

else:

    last_run_number = run_numbers[-1]


print(f"Last Run Number: {last_run_number}")

Last Run Number: 0


In [153]:
def find_distance(p1, p2, p3):

    # using the formula ax + by + c = 0
    a = p1[1] - p2[1]
    b = p2[0] - p1[0]
    c = p1[0] * p2[1] - p2[0] * p1[1]

    # use dot product to find the distance between a line and a point
    return abs( a * p3[0] + b * p3[1] + c) / math.sqrt(a * a + b * b)


def create_segment(p1, p2, v):

    above = []
    below = []

    if p2[0] - p1[0] == 0:
        return above, below
    
    #calculate m and o from y = mx + o
    m = (p2[1] - p1[1]) / (p2[0] - p1[0])
    c = -m * p1[0] + p1[1]

    #loop through each coordinate and place it into the correct list
    for coordinate in v:

        #y > mx + o means it is above the line
        if coordinate[1] > m * (coordinate[0]) + c:
            above.append(coordinate)
        #y < mx + o means it is below the line
        elif coordinate[1] < m * (coordinate[0]) + c:
            below.append(coordinate)


    return above, below



def upper_lower_hull(p1, p2, segment, flag):

    if segment == [] or p1 is None or p2 is None:
        return []

    convex_hull = []

    # calculate the distance of every point from the line to find the farthest point
    farthest_distance = -1
    farthest_point = None

    for point in segment:

        distance = find_distance(p1, p2, point)

        if distance > farthest_distance:
            
            farthest_distance = distance
            farthest_point = point
            
    convex_hull = convex_hull + [farthest_point]

    # point is now in the convex hull so remove it from the segment
    segment.remove(farthest_point)

    # determine the segments formed from two lines p1-farthest_point and p2-farthest_point
    point1above, point1below = create_segment(p1, farthest_point, segment)
    point2above, point2below = create_segment(p2, farthest_point, segment)

    # only use the segmetns in the same direction, the opposite direction is contained in the convex hull
    if flag == "above":

        convex_hull = convex_hull + upper_lower_hull(p1, farthest_point, point1above, "above")
        convex_hull = convex_hull + upper_lower_hull(farthest_point, p2, point2above, "above")

    else:

        convex_hull = convex_hull + upper_lower_hull(p1, farthest_point, point1below, "below")
        convex_hull = convex_hull + upper_lower_hull(farthest_point, p2, point2below, "below")

    return convex_hull


def quickhull(v):

    if len(v) <= 2:
        
        return v
        
    convex_hull = []

    sort = sorted(v, key = lambda x : x[0])

    p1 = sort[0]
    p2 = sort[-1]

    convex_hull = convex_hull + [p1, p2]
    
    # remove from the list as they are now in the convex hull
    sort.pop(0)
    sort.pop(-1)

    #determine points above and below the line
    above, below = create_segment(p1, p2, sort)
    
    convex_hull = convex_hull + upper_lower_hull(p1, p2, above, "above")
    
    convex_hull = convex_hull + upper_lower_hull(p1, p2, below, "below")

    # return convex_hull


def create_random_points(num_points):

    points = []
    
    for _ in range(num_points):

        x = random.uniform(0.0, 500.0)  # Adjust the range as needed
        y = random.uniform(0.0, 500.0)  # Adjust the range as needed

        points.append((x, y))

    return points

In [154]:
def main(run_number, num_points):

    # Create and open a CSV file for logging
    with open('Testfile_Quickhull.csv', 'a', newline='') as csvfile:

        fieldnames = ["Test_Case", "Number_of_Points", "Execution Time", "Execution Time on CPU"]
        
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)

        points = create_random_points(num_points)

        start_time = time.time()
        process_start_time = time.process_time()

        quickhull(points)

        process_end_time = time.process_time()
        end_time = time.time()  # Record the end time
        
        process_execution_time = process_end_time - process_start_time
        execution_time = end_time - start_time  # Calculate the execution time

        # Write the execution time to the CSV file
        writer.writerow({'Test_Case' : run_number, 'Number_of_Points': num_points, 
                         'Execution Time': f":{execution_time::.4f}",
                         'Execution Time on CPU' : f":{process_execution_time:.4f}"})
    
    # Manually close the CSV file
    csvfile.close()
        
    print(f"Test Run {run_number} is finished")


In [155]:
# start = 1_000_000_000
# end = 100_000_000_000
# step = 1_000_000_000

# numbers = list(range(start, end + 1, step))

# # Format the numbers with underscores
# formatted_numbers = [f'{n:_}' for n in numbers]
# formatted_numbers

In [156]:
num_points_array = [

    1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  # Basic Testing and boundary cases
    31, 32, 33, 34, 35, 36, 37, 38, 39, 40,  # Basic Testing
    100, 200, 300, 400, 500, 600, 700, 800, 900, 1_000, # Performance Testing
    2_000, 2_500, 3_000, 3_500, 4_000, 4_500, 5_000, 5_500, 6_000, 6_500, 7_000, 7_500, 8_000, 8_500, 9_000, 9_500, 10_000, # Performance Testing
    15_000, 20_000, 30_000, 40_000, 50_000, 60_000, 70_000, 80_000, 90_000, 100_000, # Performance Testing
    200_000, 300_000, 400_000, 500_000, 600_000, 700_000, 800_000, 900_000, 1_000_000, # Performance Testing
    1_500_000, 2_000_000, 3_000_000, 4_000_000, 5_000_000, 6_000_000, 7_000_000, 8_000_000, 9_000_000, 10_000_000, # Performance Testing
    20_000_000, 30_000_000, 40_000_000, 50_000_000, 60_000_000, 70_000_000, 80_000_000, 90_000_000, 100_000_000, # Performance Testing
    110_000_000, 120_000_000, 130_000_000, 140_000_000, 150_000_000, 160_000_000, 170_000_000, 180_000_000, 190_000_000, 200_000_000, # Performance Testing
    210_000_000, 220_000_000, 230_000_000, 240_000_000, 250_000_000, 260_000_000, 270_000_000, 280_000_000, 290_000_000, 300_000_000, # Performance Testing
    310_000_000, 320_000_000, 330_000_000, 340_000_000, 350_000_000, 360_000_000, 370_000_000, 380_000_000, 390_000_000, 400_000_000, # Performance Testing
    410_000_000, 420_000_000, 430_000_000, 440_000_000, 450_000_000, 460_000_000, 470_000_000, 480_000_000, 490_000_000, 500_000_000, # Performance Testing
    510_000_000, 520_000_000, 530_000_000, 540_000_000, 550_000_000, 560_000_000, 570_000_000, 580_000_000, 590_000_000, 600_000_000, # Performance Testing
    610_000_000, 620_000_000, 630_000_000, 640_000_000, 650_000_000, 660_000_000, 670_000_000, 680_000_000, 690_000_000, 700_000_000, # Performance Testing
    710_000_000, 720_000_000, 730_000_000, 740_000_000, 750_000_000, 760_000_000, 770_000_000, 780_000_000, 790_000_000, 800_000_000, # Performance Testing
    810_000_000, 820_000_000, 830_000_000, 840_000_000, 850_000_000, 860_000_000, 870_000_000, 880_000_000, 890_000_000, 900_000_000, # Performance Testing
    910_000_000, 920_000_000, 930_000_000, 940_000_000, 950_000_000, 960_000_000, 970_000_000, 980_000_000, 990_000_000, 1_000_000_000, # Performance Testing
    # 2_000_000_000, 3_000_000_000, 4_000_000_000, 5_000_000_000, 6_000_000_000, 7_000_000_000, 8_000_000_000, 9_000_000_000, 10_000_000_000, # Performance Testing
    # 11_000_000_000, 12_000_000_000, 13_000_000_000, 14_000_000_000, 15_000_000_000, 16_000_000_000, 17_000_000_000, 18_000_000_000, 19_000_000_000, 20_000_000_000, # Performance Testing
    # 21_000_000_000, 22_000_000_000, 23_000_000_000, 24_000_000_000, 25_000_000_000,26_000_000_000, 27_000_000_000, 28_000_000_000, 29_000_000_000, 30_000_000_000, # Performance Testing
    # 31_000_000_000, 32_000_000_000, 33_000_000_000, 34_000_000_000, 35_000_000_000, 36_000_000_000, 37_000_000_000, 38_000_000_000, 39_000_000_000, 40_000_000_000, # Performance Testing
    # 41_000_000_000, 42_000_000_000, 43_000_000_000, 44_000_000_000, 45_000_000_000, 46_000_000_000, 47_000_000_000, 48_000_000_000, 49_000_000_000, 50_000_000_000, # Performance Testing
    # 51_000_000_000, 52_000_000_000, 53_000_000_000, 54_000_000_000, 55_000_000_000, 56_000_000_000, 57_000_000_000, 58_000_000_000, 59_000_000_000, 60_000_000_000, # Performance Testing
    # 61_000_000_000, 62_000_000_000, 63_000_000_000, 64_000_000_000, 65_000_000_000, 66_000_000_000, 67_000_000_000, 68_000_000_000, 69_000_000_000, 70_000_000_000, # Performance Testing
    # 71_000_000_000, 72_000_000_000, 73_000_000_000, 74_000_000_000, 75_000_000_000, 76_000_000_000, 77_000_000_000, 78_000_000_000, 79_000_000_000, 80_000_000_000, # Performance Testing
    # 81_000_000_000, 82_000_000_000, 83_000_000_000, 84_000_000_000, 85_000_000_000, 86_000_000_000, 87_000_000_000, 88_000_000_000, 89_000_000_000, 90_000_000_000, # Performance Testing
    # 91_000_000_000, 92_000_000_000, 93_000_000_000, 94_000_000_000, 95_000_000_000, 96_000_000_000, 97_000_000_000, 98_000_000_000, 99_000_000_000, 100_000_000_000 #  Performance Testing
]

In [157]:
for num_points in num_points_array:

    last_run_number += 1 # to get the new run number

    main(last_run_number, num_points)


Test Run 1 is finished
Test Run 2 is finished
Test Run 3 is finished
Test Run 4 is finished
Test Run 5 is finished
Test Run 6 is finished
Test Run 7 is finished
Test Run 8 is finished
Test Run 9 is finished
Test Run 10 is finished
Test Run 11 is finished
Test Run 12 is finished
Test Run 13 is finished
Test Run 14 is finished
Test Run 15 is finished
Test Run 16 is finished
Test Run 17 is finished
Test Run 18 is finished
Test Run 19 is finished
Test Run 20 is finished
Test Run 21 is finished
Test Run 22 is finished
Test Run 23 is finished
Test Run 24 is finished
Test Run 25 is finished
Test Run 26 is finished
Test Run 27 is finished
Test Run 28 is finished
Test Run 29 is finished
Test Run 30 is finished
Test Run 31 is finished
Test Run 32 is finished
Test Run 33 is finished
Test Run 34 is finished
Test Run 35 is finished
Test Run 36 is finished
Test Run 37 is finished
Test Run 38 is finished
Test Run 39 is finished
Test Run 40 is finished
Test Run 41 is finished
Test Run 42 is finished
T